In [ ]:
#PyTorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

#Numpy
import numpy as np

#Dataset
import torch.utils.data as utils

#Graphs
import matplotlib.pyplot as plt

#For paths
import sys
import os
import glob

#imread and resize
from skimage import io, transform

#split dataset
from sklearn.model_selection import train_test_split

#Timestamp
import datetime

#PyTorch Models
path = os.path.join(os.path.dirname(os.path.abspath('__file__')), "models")
sys.path.append(path)
from models import *

In [ ]:
root_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), "kinect_leap_dataset", "acquisitions")
p_id = ["P1", "P2", "P3", "P4", "P5", "P6", "P7", "P8", "P9", "P10", "P11", "P12", "P13", "P14"]
g_id = ["G1", "G2", "G3", "G4", "G5", "G6", "G7", "G8", "G9", "G10"]

In [ ]:
print(os.path.join(root_path, "P1", "G1"))
files = glob.glob(os.path.join(root_path, "P1", "G1", "*rgb.png"))
print(files)
print(len(files))

In [ ]:
dataset = []
labels = []

for p in p_id:
    for g in g_id:
        path = os.path.join(root_path, p, g)
        image_names = glob.glob(os.path.join(path, "*rgb.png"))
        for img_path in image_names:
            img = io.imread(img_path)
            
            #img = transform.rescale(img, 1.0 / 20.0)
            img = transform.resize(img, (32, 32))
            #48x64x3 -> 3x48x64
            img = np.moveaxis(img, [0, 1, 2], [1, 2, 0])
            
            dataset.append(img)
            
            #label 10 will be 0
            tmp = np.zeros(10)
            tmp[int(g[-1])] = 1
            labels.append(tmp)
            #labels.append(int(g[-1]))

In [ ]:
dataset = np.array(dataset).astype(float)
labels = np.array(labels).astype(float)

In [ ]:
X_train, X_test_val, y_train, y_test_val = train_test_split(dataset, labels, test_size=0.2)
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5)

In [ ]:
dataset = torch.from_numpy(dataset).float()
labels = torch.from_numpy(labels).float()

In [ ]:
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val).float()

In [ ]:
my_dataset = utils.TensorDataset(dataset, labels) # create your dataset
my_dataloader = utils.DataLoader(my_dataset, batch_size=10, shuffle=True, num_workers=4) # create your dataloader

In [ ]:
my_dataset = utils.TensorDataset(X_train, y_train) # create your dataset
train_loader = utils.DataLoader(my_dataset, batch_size=10, shuffle=True, num_workers=4) # create your dataloader

my_dataset = utils.TensorDataset(X_val, y_val) # create your dataset
val_loader = utils.DataLoader(my_dataset, batch_size=10, shuffle=True, num_workers=4) # create your dataloader

my_dataset = utils.TensorDataset(X_test, y_test) # create your dataset
test_loader = utils.DataLoader(my_dataset, batch_size=10, shuffle=True, num_workers=4) # create your dataloader

In [ ]:
model = ResNet152()
#Target has to be LongSensor for CrossEntropyLoss
criterion = nn.CrossEntropyLoss()

#criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, betas=(0.9, 0.999), eps=1e-8, weight_decay=5e-4)

#optimizer = optim.SGD(resnet.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)

In [ ]:
def test():
    net = ResNet152()
    x = torch.randn(2,3,32,32)
    y = net(x)
    print(y.size())
test()

In [ ]:
train_loss_history = []
train_acc_history = []
val_acc_history = []
val_loss_history = []
num_epochs = 200
iter_per_epoch = len(train_loader)
#224 lines -> 112 val output for log_nth=10000
#1120000 iter
log_nth = 10

if torch.cuda.is_available():
    model.cuda()

for epoch in range(num_epochs):  # loop over the dataset multiple times
    for i, (inputs, targets) in enumerate(train_loader, 1):
        #inputs, targets = Variable(inputs.float()), Variable(targets.float())
        #for CrossEntropyLoss
        inputs, targets = Variable(inputs), Variable(targets)
        if torch.cuda.is_available():
            inputs, targets = inputs.cuda(), targets.cuda()
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        
        #loss = criterion(outputs, targets)
        #for CrossEntropyLoss
        loss = criterion(outputs, torch.max(targets, 1)[1])
        
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss_history.append(loss.data.cpu().numpy())
        if log_nth and i % log_nth == 0:
            last_log_nth_losses = train_loss_history[-log_nth:]
            train_loss = np.mean(last_log_nth_losses)
            print('[Iteration %d/%d] TRAIN loss: %.3f' % \
                        (i + epoch * iter_per_epoch,
                         iter_per_epoch * num_epochs,
                         train_loss))
            
    _, preds = torch.max(outputs, 1)
    _, target_indices = torch.max(targets, 1)

    train_acc = np.mean((preds == target_indices).data.cpu().numpy())
    train_acc_history.append(train_acc)
    if log_nth:
        print('[Epoch %d/%d] TRAIN acc/loss: %.3f/%.3f' % (epoch + 1,
                                                            num_epochs,
                                                            train_acc,
                                                            loss))
        '''_, preds = torch.max(outputs, 1)

        # Only allow images/pixels with label >= 0 e.g. for segmentation
        targets_mask = labels >= 0
        train_acc = np.mean((preds == targets)[targets_mask].data.cpu().numpy())
        train_acc_history.append(train_acc)
        if log_nth:
            print('[Epoch %d/%d] TRAIN acc/loss: %.3f/%.3f' % (epoch + 1,
                                                                   num_epochs,
                                                                   train_acc,
                                                                   train_loss))'''
        
    # VALIDATION
    val_losses = []
    val_scores = []
    model.eval()
    for inputs, targets in val_loader:
        inputs, targets = Variable(inputs), Variable(targets)
        if torch.cuda.is_available():
            inputs, targets = inputs.cuda(), targets.cuda()

        outputs = model.forward(inputs)
        
        #loss = criterion(outputs, targets)
        
        #for CrossEntropyLoss
        loss = criterion(outputs, torch.max(targets, 1)[1])
        
        val_losses.append(loss.data.cpu().numpy())

        _, preds = torch.max(outputs, 1)
        _, target_indices = torch.max(targets, 1)

        scores = np.mean((preds == target_indices).data.cpu().numpy())
        val_scores.append(scores)

    model.train()
    val_acc, val_loss = np.mean(val_scores), np.mean(val_losses)
    val_acc_history.append(val_acc)
    val_loss_history.append(val_loss)
    if log_nth:
        print('[Epoch %d/%d] VAL   acc/loss: %.3f/%.3f' % (epoch + 1,
                                                            num_epochs,
                                                            val_acc,
                                                            val_loss))

print('Finished Training')

In [ ]:
params = list(model.parameters())

In [ ]:
#https://pytorch.org/docs/stable/notes/serialization.html#recommend-saving-models
currentDT = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
path = os.path.join(os.path.dirname(os.path.abspath('__file__')), "saved_models", "resnet_rgb_" + str(num_epochs) + "_" + currentDT + ".model")
torch.save(model.state_dict(), path)

#Loading
#model = VGG('VGG16')
#model.load_state_dict(torch.load(path))

In [ ]:
#model = ResNet152()
#if torch.cuda.is_available():
#    model = model.cuda()

#path = "C:/Users/Bilal/git/gesture_recog_leap/saved_models/resnet_rgb_100_2018-07-04_06-59-19.model"
#model.load_state_dict(torch.load(path))

In [ ]:
scores = []
for inputs, target in test_loader:
    inputs, targets = Variable(inputs), Variable(target)
    if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    _, target_indices = torch.max(targets, 1)
    scores.extend((preds == target_indices).data.cpu().numpy())
    
print('Test set accuracy: %f' % np.mean(scores))

In [ ]:
plt.plot(train_loss_history, '-')
#plt.plot(val_loss_history, 'o')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()

plt.plot(train_acc_history, '-o')
plt.plot(val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

In [ ]:
for inputs, target in test_loader:
    inputs, targets = Variable(inputs), Variable(target)
    if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    _, target_indices = torch.max(targets, 1)
    scores.extend((preds == target_indices).data.cpu().numpy())

In [ ]:
for inputs, target in test_loader:
    inputs, targets = Variable(inputs), Variable(target)
    if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    _, target_indices = torch.max(targets, 1)
    
    numpy_inputs = inputs.data.cpu().numpy()
    numpy_outputs = outputs.data.cpu().numpy()
    numpy_targets = targets.data.cpu().numpy()
    
    img = numpy_inputs[0]
    img = img[0, :, :]
    plt.imshow(img)
    plt.title("Predicted: "+str(preds[0].item()) + " Target: "+str(target_indices[0].item()))
    plt.show()
    #currentDT = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    #path = "images/img_" + currentDT + ".png"
    #plt.imsave(path, img.astype(float))